In [1]:
import cv2 
import math
import numpy as np
import imutils
from numba import jit, cuda
from timeit import default_timer as timer
import math as m
from PIL import Image 
from IPython.display import clear_output

from tensorflow import keras
import keras.utils as image

In [2]:
def classify_cashew(cam=0):
    mlt=10000
    cap = cv2.VideoCapture(int(cam),cv2.CAP_DSHOW)
    model = keras.models.load_model('cashew_320_400.h5')
    class_indices=['320','400']
    while(True):
        ret, img = cap.read()
        img=img[80:,125:-50]
        cv2.imwrite('snapshot.png', mask(img))
        
        cnn_file = 'snapshot.png'
        cnn_img = image.load_img(cnn_file, target_size=(150, 150))
        cnn_img = image.img_to_array(cnn_img)
        cnn_img = np.expand_dims(cnn_img, axis=0)
        cnn_img = cnn_img/255
        prediction_prob = model.predict(cnn_img,verbose=0)
        print(prediction_prob[0][0]*mlt)
        if prediction_prob[0][0]*mlt < 0.5:
            print(class_indices[0])
            img = cv2.putText(img,class_indices[0], (50, 50),cv2.FONT_HERSHEY_SIMPLEX,1,(0, 0, 255), 2, cv2.LINE_AA)
        else:
            print(class_indices[1])
        
        cv2.namedWindow("original", cv2.WINDOW_NORMAL)
        cv2.resizeWindow("original", 330, 255)
        cv2.namedWindow("processed1", cv2.WINDOW_NORMAL)        
        cv2.resizeWindow("processed1", 330, 255)
        # cv2.namedWindow("processed2", cv2.WINDOW_NORMAL)        
        # cv2.resizeWindow("processed2", 330, 255)
        # cv2.namedWindow("processed3", cv2.WINDOW_NORMAL)        
        # cv2.resizeWindow("processed3", 330, 255)
        # cv2.namedWindow("processed4", cv2.WINDOW_NORMAL)        
        # cv2.resizeWindow("processed4", 330, 255)

        cv2.imshow('original', img)
        cv2.imshow('processed1', mask(img))
        # cv2.imshow('processed2', fg)
        # cv2.imshow('processed3', target)
        # cv2.imshow('processed4', fimg)
        

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        clear_output(wait=True)

        # cv2.waitKey(0)
    cap.release()
    cv2.destroyAllWindows()

In [3]:
def mask(img):
    image=cv2.blur(img, (10,10))
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, (23,38,100), (40, 200, 200))
    target = cv2.bitwise_and(img,img, mask=mask)
    return target

def thresh(img):
    ret, thresh = cv2.threshold(img, 120, 255, cv2.THRESH_BINARY)
    # ret, thresh = cv2.threshold(img, 120, 255, cv2.THRESH_BINARY_INV)
    # ret, thresh = cv2.threshold(img, 120, 255, cv2.THRESH_TRUNC)
    # ret, thresh = cv2.threshold(img, 120, 255, cv2.THRESH_TOZERO)
    # ret, thresh = cv2.threshold(img, 120, 255, cv2.THRESH_TOZERO_INV)
    return thresh

In [8]:
classify_cashew(0)

9999.994039535522
400


In [199]:
class_indices={'320': 0, '400': 1}
class_indices['320']

0

In [1]:
import cv2
import numpy as np

def mask(img):
    image=cv2.blur(img, (10,10))
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    mask1 = cv2.bitwise_not(cv2.inRange(hsv, (0,0,255-150), (255,150,255)))
    mask2 = cv2.inRange(hsv, (60 - 200, 100, 100),(60 + 200, 255, 255))
    mask3 = cv2.inRange(hsv, (16, 63, 0),(24, 200, 200))
    mask = cv2.bitwise_or(cv2.bitwise_or(mask1,mask2),mask3)
    
    # mask = cv2.inRange(hsv, (23,38,100), (40, 200, 200))
    target = cv2.bitwise_and(img,img, mask=mask)
    return (target,mask)

def thresh(img):
    ret, thresh = cv2.threshold(img, 120, 255, cv2.THRESH_BINARY)
    return thresh

def threshi(img):
    # ret, thresh = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    ret, thresh = cv2.threshold(img, 120, 255, cv2.THRESH_BINARY_INV)
    return thresh

def seg(image):
    mask = np.zeros(image.shape[:2], np.uint8)
    backgroundModel = np.zeros((1, 65), np.float64)
    foregroundModel = np.zeros((1, 65), np.float64)
    rectangle = (0,0,image.shape[0],image.shape[1])
    cv2.grabCut(image, mask, rectangle,backgroundModel, foregroundModel,3, cv2.GC_INIT_WITH_RECT)
    mask2 = np.where((mask == 2)|(mask == 0), 0, 1).astype('uint8')
    image = image * mask2[:, :, np.newaxis]
    return image
cam = cv2.VideoCapture(0,cv2.CAP_DSHOW)
        
# cv2.namedWindow("original", cv2.WINDOW_NORMAL)
# cv2.resizeWindow("original", 630, 455)
# cv2.namedWindow("processed", cv2.WINDOW_NORMAL)        
# cv2.resizeWindow("processed", 630, 455)
# cv2.namedWindow("processed1", cv2.WINDOW_NORMAL)        
# cv2.resizeWindow("processed1", 630, 455)
# cv2.namedWindow("processed2", cv2.WINDOW_NORMAL)        
# cv2.resizeWindow("processed2", 630, 455)
# cv2.namedWindow("processed3", cv2.WINDOW_NORMAL)        
# cv2.resizeWindow("processed3", 330, 255)

while(True):
    # print(3)
    ret, frame = cam.read()
    # print(4)
    frame=frame[:,125:-50]
    msk,bg=mask(frame)
    gframe = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gmsk = cv2.cvtColor(msk, cv2.COLOR_BGR2GRAY)
    cv2.imshow('original', frame)
    cv2.imshow('processed', msk)
    cv2.imshow('processed1', bg)
    # cv2.imshow('processed2', threshi(gframe))
    # im=edges(frame)
    # im,lim,p=contour(frame)
    # print(p)
    # cv2.imshow('frame1', im)
    # cv2.imwrite('edge.jpg',im)
    # cv2.imwrite('piece.jpg',frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    # clear_output(wait=True)
    
# cv2.waitKey(0)
cam.release()
cv2.destroyAllWindows()

TypeError: 'NoneType' object is not subscriptable

In [2]:
def mask(img):
    image=cv2.blur(img, (10,10))
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, (23,38,100), (36, 200, 200))
    target = cv2.bitwise_and(img,img, mask=mask)